## 带有 Whisper 的交互式 Phi 3 Mini 4K Instruct 聊天机器人

### 介绍：
交互式 Phi 3 Mini 4K Instruct 聊天机器人是一款工具，允许用户通过文本或音频输入与微软 Phi 3 Mini 4K Instruct 演示进行互动。该聊天机器人可用于多种任务，如翻译、天气更新和一般信息收集。

In [ ]:
# 安装所需的 Python 库
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# 检查 Cuda 支持是否可用 
# 输出 True = Cuda
# 输出 False = 无 Cuda（需要安装 Cuda 才能在 GPU 上运行模型）
import os 
import torch
print(torch.cuda.is_available())


[创建您的 Huggingface 访问令牌](https://huggingface.co/settings/tokens)

1. 创建新令牌 
2. 提供新名称 
3. 选择写入权限
4. 复制令牌并保存在安全的地方

以下 Python 代码主要执行了两个任务：导入 `os` 模块和设置环境变量。

1. 导入 `os` 模块：
   - Python 中的 `os` 模块提供了一种与操作系统交互的方式。它允许你执行各种与操作系统相关的任务，例如访问环境变量、处理文件和目录等。
   - 在这段代码中，使用 `import` 语句导入了 `os` 模块。此语句使得 `os` 模块的功能在当前 Python 脚本中可用。

2. 设置环境变量：
   - 环境变量是一个可以被操作系统上运行的程序访问的值。它是一种存储配置设置或其他信息的方法，可以被多个程序使用。
   - 在这段代码中，使用 `os.environ` 字典设置了一个新的环境变量。字典的键是 `'HF_TOKEN'`，值是从 `HUGGINGFACE_TOKEN` 变量中获取的。
   - `HUGGINGFACE_TOKEN` 变量定义在这段代码片段的上方，并使用 `#@param` 语法赋值为字符串 `"hf_**************"`。这种语法通常用于 Jupyter notebooks，以便在 notebook 界面中直接进行用户输入和参数配置。
   - 通过设置 `'HF_TOKEN'` 环境变量，它可以被程序的其他部分或在同一操作系统上运行的其他程序访问。

总的来说，这段代码导入了 `os` 模块，并使用在 `HUGGINGFACE_TOKEN` 变量中提供的值设置了一个名为 `'HF_TOKEN'` 的环境变量。

In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN'] = HUGGINGFACE_TOKEN

In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

使用 Edge TTS 服务进行文本转语音（TTS）。让我们逐一了解相关的功能实现：

1. `calculate_rate_string(input_value)`：此函数接受一个输入值，并计算 TTS 语音的速率字符串。输入值表示所需的语速，其中值 1 表示正常速度。该函数通过从输入值中减去 1，将其乘以 100，然后根据输入值是否大于或等于 1 来确定符号。函数返回格式为 “{sign}{rate}” 的速率字符串。

2. `make_chunks(input_text, language)`：此函数接受一个输入文本和一种语言作为参数。它根据特定语言的规则将输入文本拆分为块。在此实现中，如果语言是“English”，函数将在每个句号（“.”）处拆分文本，并删除任何前导或尾随空格。然后，它将句号附加到每个块上，并返回过滤后的块列表。

3. `tts_file_name(text)`：此函数根据输入文本生成 TTS 音频文件的文件名。它对文本执行若干转换操作：去除尾随句号（如果存在），将文本转换为小写，去除前导和尾随空格，并将空格替换为下划线。然后，它将文本截断为最多 25 个字符（如果更长）或使用完整文本（如果为空）。最后，它使用 [`uuid`] 模块生成一个随机字符串，并将其与截断后的文本结合起来，创建格式为 “/content/edge_tts_voice/{truncated_text}_{random_string}.mp3” 的文件名。

4. `merge_audio_files(audio_paths, output_path)`：此函数将多个音频文件合并为一个音频文件。它接受一个音频文件路径列表和一个输出路径作为参数。函数初始化一个空的 `AudioSegment` 对象，名为 [`merged_audio`]。然后，它遍历每个音频文件路径，使用 `pydub` 库中的 `AudioSegment.from_file()` 方法加载音频文件，并将当前音频文件附加到 [`merged_audio`] 对象中。最后，它将合并后的音频以 MP3 格式导出到指定的输出路径。

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`：此函数使用 Edge TTS 服务执行 TTS 操作。它接受一个文本块列表、语速、语音名称和保存路径作为参数。如果块的数量大于 1，函数会创建一个目录来存储各个块的音频文件。然后，它遍历每个块，使用 `calculate_rate_string()` 函数、语音名称和块文本构建一个 Edge TTS 命令，并使用 `os.system()` 函数执行该命令。如果命令执行成功，它会将生成的音频文件路径附加到列表中。在处理完所有块之后，它使用 `merge_audio_files()` 函数合并各个音频文件，并将合并后的音频保存到指定的保存路径。如果只有一个块，它会直接生成 Edge TTS 命令并将音频保存到保存路径。最后，它返回生成的音频文件的保存路径。

6. `random_audio_name_generate()`：此函数使用 [`uuid`] 模块生成一个随机音频文件名。它生成一个随机 UUID，将其转换为字符串，取前 8 个字符，附加 “.mp3” 扩展名，并返回随机音频文件名。

7. `talk(input_text)`：此函数是执行 TTS 操作的主要入口点。它接受一个输入文本作为参数。首先，它检查输入文本的长度，以确定是否为长句子（大于或等于 600 个字符）。根据长度和 `translate_text_flag` 变量的值，它确定语言并使用 `make_chunks()` 函数生成文本块列表。然后，它使用 `random_audio_name_generate()` 函数生成音频文件的保存路径。最后，它调用 `edge_free_tts()` 函数执行 TTS 操作，并返回生成的音频文件的保存路径。

总体而言，这些函数协同工作，将输入文本拆分为块，生成音频文件的文件名，使用 Edge TTS 服务执行 TTS 操作，并将各个音频文件合并为一个音频文件。

In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # 创建一个空 AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # 遍历每个音频文件的路径
    for audio_path in audio_paths:
        # 用 pydub 打开音频文件
        audio = AudioSegment.from_file(audio_path)

        # 将当前音频文件添加到 merged_audio
        merged_audio += audio

    # 将合并后的音频文件导出到指定路径
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# 只需将声音性别从男性改为女性，然后选择您想使用的声音即可
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

实现两个函数：`convert_to_text` 和 `run_text_prompt`，以及声明两个类：`str` 和 `Audio`。

`convert_to_text` 函数接收一个 audio_path 作为输入，并使用一个名为 whisper_model 的模型将音频转录为文本。该函数首先检查 `gpu` 标志是否设置为 `True`。如果是，则使用 whisper_model，并设置参数如 `word_timestamps=True`, `fp16=True`, `language='English'`, 和 `task='translate'`。如果 `gpu` 标志为 `False`，则使用 `fp16=False` 的 whisper_model。然后将生成的转录文本保存到名为 'scan.txt' 的文件中，并返回该文本。

`run_text_prompt` 函数接收一个消息和一个 chat_history 作为输入。它使用 `phi_demo` 函数根据输入消息生成一个聊天机器人响应。生成的响应随后传递给 `talk` 函数，该函数将响应转换为一个音频文件并返回文件路径。`Audio` 类用于显示和播放音频文件。音频通过 `IPython.display` 模块中的 `display` 函数显示，并使用 `autoplay=True` 参数创建 `Audio` 对象，这样音频会自动开始播放。chat_history 会更新输入消息和生成的响应，并返回一个空字符串和更新后的 chat_history。

`Audio` 类是一个自定义类，表示一个音频对象。它用于在 Jupyter Notebook 环境中创建一个音频播放器。该类接受各种参数，如 `data`, `filename`, `url`, `embed`, `rate`, `autoplay` 和 `normalize`。`data` 参数可以是一个 numpy 数组、一个样本列表、一个表示文件名或 URL 的字符串，或原始 PCM 数据。`filename` 参数用于指定要从中加载音频数据的本地文件，`url` 参数用于指定要从中下载音频数据的 URL。`embed` 参数决定音频数据是应该使用数据 URI 嵌入还是从原始来源引用。rate 参数指定音频数据的采样率。`autoplay` 参数决定音频是否应自动开始播放。`normalize` 参数指定音频数据是否应归一化（重新缩放到最大可能范围）。Audio 类还提供像 `reload` 这样的方法，用于从文件或 URL 重新加载音频数据，以及像 `src_attr`, `autoplay_attr` 和 `element_id_attr` 这样的属性，用于检索 HTML 中音频元素的相应属性。

总体来说，这些函数和类用于将音频转录为文本，从聊天机器人生成音频响应，并在 Jupyter Notebook 环境中显示和播放音频。

In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)